In [ ]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

In [ ]:

def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
  
def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
  
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]  
    
    #здесь получаем макс gain
    node = find_winner(df)
    #его значение
    attValue = np.unique(df[node])
        
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    #рекурсивно создаем дерево

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Урожайность фактическая '],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree
  


In [ ]:
#/content/620abc503937463d.xlsx
file = pd.read_excel("/content/620abc503937463d.xlsx")
df = pd.DataFrame(file)
df = df.drop(columns=['Дата'])

In [ ]:
df

,Вождь,Время года,Температура,Влажность,Сколько людей занято посевами,Предсказания шамана,Урожайность фактическая
0,Воин,лето1,Низкая,Низкая,Меньше половины,Низкая,Низкая
1,Воин,лето2,Высокая,Низкая,Меньше половины,Низкая,Высокая
2,Фермер,лето1,Средняя,Высокая,Больше половины,Высокая,Высокая
3,Фермер,лето2,Высокая,Средняя,Больше половины,Средняя,Высокая
4,Путешественник,лето2,Средняя,Средняя,Меньше половины,Высокая,Средняя
5,Путешественник,лето1,Низкая,Низкая,Меньше половины,Низкая,Низкая
6,Воин,лето1,Высокая,Высокая,Меньше половины,Низкая,Низкая
7,Воин,лето2,Средняя,Высокая,Меньше половины,Низкая,Средняя
8,Фермер,лето1,Средняя,Средняя,Больше половины,Средняя,Высокая
9,Фермер,лето2,Средняя,Низкая,Больше половины,Средняя,Средняя


In [ ]:
t = buildTree(df)

In [ ]:
import pprint
pprint.pprint(t)

{'Температура': {'Высокая': {'Время года': {'лето1': {'Влажность': {'Высокая': 'Низкая',
                                                                    'Средняя': 'Средняя'}},
                                            'лето2': 'Высокая'}},
                 'Низкая': 'Низкая',
                 'Средняя': {'Вождь': {'Воин': 'Средняя',
                                       'Путешественник': 'Средняя',
                                       'Фермер': {'Влажность': {'Высокая': 'Высокая',
                                                                'Низкая': 'Средняя',
                                                                'Средняя': 'Высокая'}}}}}}
